## Segmentation

* https://paper.dropbox.com/doc/Segmentation-RpcEF8PYcnEjbW35LP41a

In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import os
import sys
p = os.path.join(os.path.dirname('__file__'), '..')
sys.path.append(p)
from common import *
from matplotlib.pyplot import Rectangle

## Data

In [ ]:
DATA_DIR = '../data/'
json_fpath = os.path.join(DATA_DIR, 'volleyball_frame_00665.json')
img_fpath = os.path.join(DATA_DIR, 'volleyball_frame_00665.png')
IMG_DIR = os.path.join(DATA_DIR, 'volleyball', 'images_subset')
metadata_fpath = os.path.join(DATA_DIR, 'volleyball_bbox_labels.csv')

## Helpers

In [ ]:
def plot_img(arr, fs=(10,10), cmap='gray', title=None):
    plt.figure(figsize=fs)
    plt.imshow(arr, cmap=cmap)
    plt.title(title)
    plt.show()
    
def load_img(fpath):
    return plt.imread(fpath)

def load_cv2_img(fpath, w=None, h=None, colorspace=None):
    img = cv2.imread(img_fpath)
    if colorspace is not None:
        img = cv2.cvtColor(img, colorspace)
    if None not in [w,h]:
        img = cv2.resize(img, (w, h), interpolation=cv2.INTER_CUBIC)
    return img

def threshold_color(img, color, thresh, sigma=1.0):
    """
    color = [b, g, r] or [r,b,g] or [h,s,v]
    thresh = [b,g,r] margin allowed around color (1 per channel)
    """
    if isinstance(thresh, int):
        thresh = [thresh]*3
    thresh = np.array(thresh) * sigma
    min_color = np.array([color[0]-thresh[0], color[1]-thresh[1], color[2]-thresh[2]])
    max_color = np.array([color[0]+thresh[0], color[1]+thresh[1], color[2]+thresh[2]])
    min_color[min_color < 0] = 0
    max_color[max_color > 255] = 255
    print("Min", min_color)
    print("Max", max_color)
    
    mask = cv2.inRange(img, min_color, max_color)
    result = cv2.bitwise_and(img, img, mask=mask)
    return mask, result

def get_color_of_pixel(fpath, x, y, colorspace='BGR'):
    rgb_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
    hsv_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)
    
    rgb_colors = rgb_img[y,x,:]
    hsv_colors = hsv_img[y,x,:]
    bgr_colors = np.copy(rgb_colors[::-1])
    print ("BGR:", bgr_colors)
    print ("RGB:", rgb_colors)
    print ("HSV:", hsv_colors)
    
    # Plot to visualize
    img = np.copy(rgb_img)
    img[y-5:y+5:,x-5:x+5,:] = 255
    img[y,x,:] = 0
    plot_img(img, fs=(18,18))
    
    if colorspace == 'BGR':
        return bgr_colors.tolist()
    if colorspace == 'RGB':
        return rgb_colors.tolist()
    return hsv_colors.tolist()

def get_hsv_value_of_bgr(bgr_color):
    print("BGR", bgr_color)
    bgr_color = np.uint8([[bgr_color]])
    hsv = cv2.cvtColor(bgr_color, cv2.COLOR_BGR2HSV)[0][0]
    print("HSV", hsv)
    return hsv

# Create color histograms representing average values among samples
def get_flattened_channels(imgs):
    chans = np.empty(shape=(1,3))
    for img in imgs:
        h,w,c = img.shape
        reshaped = img.reshape((h*w, c))
        chans = np.concatenate([chans, reshaped], axis=0)
        #print(reshaped.shape, chans.shape)
    return chans

# Plotting Histogram (all channels flattened)
def plot_hist(img, bins=256, title=None):
    plt.hist(img.ravel(), bins=bins, range=[0,256])
    plt.title(title)
    plt.show()

def plot_bgr_hist(bgr_img, bins=256, mask=None):
    # Mask let's you select for certain regions    
    color = ('b','g','r')
    for i,col in enumerate(color):
        histr = cv2.calcHist([bgr_img],[i],mask,[bins],[0,256])
        plt.plot(histr, color=col)
        plt.xlim([0,bins])
    plt.show()
    
def plot_bbs_from_rectLabel_annos(json_fpath, img_fpath):
    bb_json = json.load(open(json_fpath, 'r'))
    fig = plt.figure(figsize=(18,18))
    axes = plt.axes([0, 0.03, 1, 0.97])
    
    img = plt.imread(img_fpath)
    imgplot = axes.imshow(img)

    for box in bb_json['objects']:
        label = box['label']
        color = BOX_COLORS[label]
        coords = box['x_y_w_h']
        bb = Rectangle(
            (coords[0],coords[1]), 
            coords[2], coords[3],
            fill=False,
            edgecolor=color,
            linewidth=2)
        axes.add_patch(bb)
        
def get_img_crops_from_rectLabel_bbs(img, json_fpath):
    crops_dict = {}
    bb_json = json.load(open(json_fpath, 'r'))
    for box in bb_json['objects']:
        label = box['label']
        x,y,w,h = box['x_y_w_h']
        crop = img[y:y+h,x:x+w,:]
        if label not in crops_dict:
            crops_dict[label] = []
        crops_dict[label].append(crop)
    return crops_dict

## Color Histograms

In [ ]:
bgr_img = load_cv2_img(img_fpath)

# Cv2 Histogram (faster)
hist = cv2.calcHist(images=[bgr_img], channels=[0], mask=None, histSize=[256], ranges=[0,256])

# Numpy Histogram (slower)
hist, bins = np.histogram(bgr_img.ravel(), 256, [0,256])

plot_hist(bgr_img)
plot_bgr_hist(bgr_img)

## BGR Thresholding

In [ ]:
# Highlight the court
bgr_color = get_color_of_pixel(img_fpath, 700, 650, 'BGR')
bgr_img = load_cv2_img(img_fpath)
print(bgr_color)
mask, result = threshold_color(bgr_img, bgr_color, 35)
plot_img(result)

In [ ]:
# Highlight floor around court (darker green)
bgr_color = get_color_of_pixel(img_fpath, 1100, 550, 'BGR')
bgr_img = load_cv2_img(img_fpath)
mask1, result1 = threshold_color(bgr_img, bgr_color, 60)
plot_img(result1)

In [ ]:
# Highlight floor around court (lighter green reflection)
bgr_color = get_color_of_pixel(img_fpath, 1130, 460, 'BGR')
bgr_img = load_cv2_img(img_fpath)
mask2, result2 = threshold_color(bgr_img, bgr_color, 50)
plot_img(result2)

### HSV Thresholding

In [ ]:
hsv_color = get_color_of_pixel(img_fpath, 700, 650, 'HSV')
hsv_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)
mask, result = threshold_color(hsv_img, hsv_color, 35)
plot_img(result)

In [ ]:
hsv_color = get_color_of_pixel(img_fpath, 1130, 460, 'HSV')
hsv_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)
mask, result = threshold_color(hsv_img, hsv_color, 55)
plot_img(result)

In [ ]:
hsv_color = get_color_of_pixel(img_fpath, 25, 550, 'HSV')
hsv_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)
mask, result = threshold_color(hsv_img, hsv_color, 55)
plot_img(result)

## Data Driven Thresholding

* https://gist.github.com/danielballan/ab5e28420ba1b24c5ad4

In [ ]:
json_fpath = os.path.join(DATA_DIR, 'volleyball_frame_00665.json')
img_fpath = os.path.join(DATA_DIR, 'volleyball_frame_00665.png')

BOX_COLORS = {
    'referee': 'black',
    'red_team': 'red',
    'blue_team': 'blue',
    'court-inner': 'green',
    'court_outer': 'white',
}

plot_bbs_from_rectLabel_annos(json_fpath, img_fpath)

In [ ]:
bgr_img = load_cv2_img(img_fpath)
crops = get_img_crops_from_rectLabel_bbs(bgr_img, json_fpath)

In [ ]:
for crop in crops['red_team']:
    plot_img(crop)

In [ ]:
for crop in crops['court-inner']:
    plot_img(crop)

In [ ]:
# BGR Histograms
bgr_img = load_cv2_img(img_fpath)
crops = get_img_crops_from_rectLabel_bbs(bgr_img, json_fpath)

hists = {}
for label in crops.keys():
    chans = get_flattened_channels(crops[label])
    bgr = ('b','g','r')
    hists[label] = {
        'b':None,
        'g':None,
        'r':None
    }
    for i in range(len(chans[0])):
        hist, bins = np.histogram(chans[:,i], 50, [0,256])
        hists[label][bgr[i]] = hist
        plot_hist(chans[:,i], bins=50, title=label + ' ' + bgr[i])

### RGB Threshold

In [ ]:
# RGB

img_fpath = os.path.join(IMG_DIR, 'volleyball_frame_00665.png')
img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2RGB)
plot_img(img, fs=(14,14), title="Original")

r = 175 + 220 // 2
g = 90 + 135 // 2
b = 75 + 125 // 2
color = (
    r,g,b
)
thresh = (
    220 - 175 // 2,
    135 - 90 // 2,
    125 - 75 // 2
)
mask, result = threshold_color(img, color, thresh, sigma=.9)
plot_img(result, fs=(14,14), title="Thresholded")

## HSV Threshold

In [ ]:
img_fpath = os.path.join(IMG_DIR, 'volleyball_frame_00665.png')
hsv_img = load_cv2_img(img_fpath, colorspace=cv2.COLOR_BGR2HSV)
plot_img(hsv_img, fs=(14,14), title="Original")

In [ ]:
crops = get_img_crops_from_rectLabel_bbs(hsv_img, json_fpath)
hists = {}
for label in crops.keys():
    chans = get_flattened_channels(crops[label])
    channels = ('h','s','v')
    hists[label] = {c:None for c in bgr}
    for i in range(len(chans[0])):
        hist, bins = np.histogram(chans[:,i], 50, [0,256])
        hists[label][bgr[i]] = hist
        plot_hist(chans[:,i], bins=50, title=label + ' ' + channels[i])

In [ ]:
"""
H 0 - 10
S = 100 - 150
V = 175 - 220
"""
color = (
    5,
    50,
    45
)
thresh = (
    6,
    256,
    256
)
mask, result = threshold_color(img, color, thresh, sigma=1)
plot_img(result, fs=(14,14), title="Thresholded")

## Color Quantization

* https://docs.opencv.org/3.0-beta/doc/py_tutorials/py_ml/py_kmeans/py_kmeans_opencv/py_kmeans_opencv.html


In [ ]:
img = cv2.imread(img_fpath)
hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
Z = img.reshape((-1,3))

# convert to np.float32
Z = np.float32(Z)

# define criteria, number of clusters(K) and apply kmeans()
criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 10, 1.0)
K = 6
ret, label, center = cv2.kmeans(Z,K,None,criteria,10,cv2.KMEANS_RANDOM_CENTERS)

# Now convert back into uint8, and make original image
center = np.uint8(center)
res = center[label.flatten()]
res2 = res.reshape((img.shape))

plot_img(res2, fs=(12,8))

## Contours

In [ ]:
h,w,c = img.shape
minarea = h * w / 10
gray = cv2.GaussianBlur(res2, ksize=(5,5), sigmaX=3)
gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)

ret,thresh = cv2.threshold(gray, 127, 255, 0)
image, contours, hierarchy = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
img = cv2.drawContours(img, contours, -1, (0,255,0), 3)

In [ ]:
for cnt in contours:
    if cv2.contourArea(cnt) < minarea:
        img = cv2.drawContours(img, [cnt], -1, (0,0,0), 3)


In [ ]:
plot_img(img, fs=(20,15))

## Links